In [9]:
import os
import zipfile
import shutil
import pandas as pd
from tqdm import tqdm

In [30]:
########### W MIEJSCU ... PODAJ ŚCIEŻKĘ DO FOLDERU, W KTÓRYM ZNAJDUJĄ SIĘ DANE FITBITA ##########
main_path = "C:/.../fitbit/fitbit_in"

#main_folder = "./dataset/fitbit/"

# Funkcje do czyszczenia folderów

Funkcje do wypakowywania folderów, usuwania niepotrzebnych plików (readme) i pustych plików.

In [10]:
def unpack_zip_folder(main_path):
    for root, __, users_folders in os.walk(main_path):
        if len(users_folders) > 0:
            # dla każdego usera
            iter_folder = main_path+root[root.find('iter'):]
            for user in users_folders:
                # określanie ścieżek do plików
                zip_path = iter_folder+'/'+user
                extract_path = iter_folder+'/'+str(user.split('.zip')[0])

                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_path)

In [11]:
def remove_readme_files(path):
    for f in os.listdir(path):
        if f.lower().find("readme") != -1 or f.lower().find('.pdf') != -1 or f.lower().find(".txt") != -1:
            full_path = os.path.join(path, f)
            os.remove(full_path)

In [12]:
def remove_empty_files(path):
    for f in os.listdir(path):
        if f.lower().find(".csv") != -1:
            # wczytywanie pliku
            full_path = os.path.join(path, f)
            try:
                data = pd.read_csv(full_path)
                # jeżeli plik jest pusty, to go usuwamy
                if len(data) == 0:
                    os.remove(full_path)
            except:
                pass

In [13]:
def remove_empty_folders(path):
    if len(os.listdir(path)) == 0:
        shutil.rmtree(path)

# Funkcje do agregacji plików

Funkcja do zwracania listy plików w danym folderze.

In [14]:
def return_files_list(path):
    files_list = []
    for f in os.listdir(path):
        files_list.append(f)    
    return files_list

Funkcje do wczytywania i zapisywania DataFramów z podanych plików (każda funkcja osobno dla każdego folderu).

In [15]:
def create_other_dataFrames(path):
    df_oxygen = pd.DataFrame()
    for f in os.listdir(path):
        full_path = os.path.join(path, f)
        if f.find('estimated_oxygen_variation') != -1:
            data = pd.read_csv(full_path)
            df_oxygen = df_oxygen.append(data, ignore_index=True)
    return [df_oxygen]

In [16]:
def create_heart_dataFrames(path):
    df_alfib = pd.DataFrame()
    df_alerts = pd.DataFrame()
    df_profile = pd.DataFrame()
    for f in os.listdir(path):
        full_path = os.path.join(path, f)
        if (f.find('afib_ecg_reading_') != -1) or (f.find('Afib ECG Readings') != -1):
            data = pd.read_csv(full_path)
            df_alfib = df_alfib.append(data, ignore_index=True)
        elif f.find('Heart Rate Notifications Alerts') != -1:
            data = pd.read_csv(full_path)
            df_alerts = df_alerts.append(data, ignore_index=True)
        elif f.find('Heart Rate Notifications Profile') != -1:
            data = pd.read_csv(full_path)
            df_profile = df_profile.append(data, ignore_index=True)
    return df_alfib, df_alerts, df_profile

In [17]:
def create_physical_activity_dataFrames_part1(path):
    df_active_zone = pd.DataFrame()
    df_goals = pd.DataFrame()
    df_calories = pd.DataFrame()
    df_demographic = pd.DataFrame()
    df_distance = pd.DataFrame()
    df_exercise = pd.DataFrame()

    for f in os.listdir(path):
        full_path = os.path.join(path, f)
        if f.find('Active Zone Minutes') != -1:
            data = pd.read_csv(full_path)
            df_active_zone = df_active_zone.append(data, ignore_index=True)
        elif f.find('Activity Goals') != -1:
            data = pd.read_csv(full_path)
            df_goals = df_goals.append(data, ignore_index=True)
        elif f.find('calories') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_calories = df_calories.append(data, ignore_index=True)
        elif f.find('demographic') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_demographic = df_demographic.append(data, ignore_index=True)
        elif f.find('distance') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_distance = df_distance.append(data, ignore_index=True)
        elif f.find('exercise') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_exercise = df_exercise.append(data, ignore_index=True)
        
    return df_active_zone, df_goals, df_calories, df_demographic, df_distance, df_exercise, 

In [18]:
def create_physical_activity_dataFrames_part2(path):
    df_heart_rate = pd.DataFrame()
    df_lightly_active_minutes = pd.DataFrame()
    for f in os.listdir(path):
        full_path = os.path.join(path, f)
        if f.find('heart_rate-') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_heart_rate = df_heart_rate.append(data, ignore_index=True)
        elif f.find('lightly_active_minutes') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_lightly_active_minutes = df_lightly_active_minutes.append(data, ignore_index=True)
    return df_heart_rate, df_lightly_active_minutes

In [19]:
def create_physical_activity_dataFrames_part3(path):
    df_moderately_active_minutes = pd.DataFrame()
    df_resting_heart_rate = pd.DataFrame()
    df_sedentary_minutes = pd.DataFrame()
    df_steps = pd.DataFrame()
    df_swim_lengths_data = pd.DataFrame()
    for f in os.listdir(path):
        full_path = os.path.join(path, f)
        if f.find('moderately_active_minutes') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_moderately_active_minutes = df_moderately_active_minutes.append(data, ignore_index=True)
        elif f.find('resting_heart_rate') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_resting_heart_rate = df_resting_heart_rate.append(data, ignore_index=True)
        elif f.find('sedentary_minutes') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_sedentary_minutes = df_sedentary_minutes.append(data, ignore_index=True)
        elif f.find('steps') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_steps = df_steps.append(data, ignore_index=True)
        elif f.find('swim_lengths_data') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_swim_lengths_data = df_swim_lengths_data.append(data, ignore_index=True)

    return df_moderately_active_minutes, df_resting_heart_rate, df_sedentary_minutes, df_steps, df_swim_lengths_data

In [20]:
def create_physical_activity_dataFrames_part4(path):
    df_time_in_heart_rate_zones = pd.DataFrame()
    df_very_active_minutes = pd.DataFrame()
    for f in os.listdir(path):
        full_path = os.path.join(path, f)        
        if f.find('time_in_heart_rate_zones') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_time_in_heart_rate_zones = df_time_in_heart_rate_zones.append(data, ignore_index=True)
        elif f.find('very_active_minutes') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_very_active_minutes = df_very_active_minutes.append(data, ignore_index=True)
    return df_time_in_heart_rate_zones, df_very_active_minutes

In [21]:
def create_sleep_dataFrames(path):
    df_temp = pd.DataFrame()
    df_var_summary = pd.DataFrame()
    df_drrs = pd.DataFrame()
    df_daily_spo2 = pd.DataFrame()
    df_device_temp = pd.DataFrame()
    df_hr_var = pd.DataFrame()
    df_hr_hist = pd.DataFrame()
    df_min_spo2 = pd.DataFrame()
    df_rrs = pd.DataFrame()
    df_sleep = pd.DataFrame()
    df_sleep_score = pd.DataFrame()
    for f in os.listdir(path):
        full_path = os.path.join(path, f)
        if f.find('Computed Temperature') != -1:
            data = pd.read_csv(full_path)
            df_temp = df_temp.append(data, ignore_index=True)
        elif f.find('Daily Heart Rate Variability Summary') != -1:
            data = pd.read_csv(full_path)
            df_var_summary = df_var_summary.append(data, ignore_index=True)
        elif f.find('Daily Respiratory Rate Summary') != -1:
            data = pd.read_csv(full_path)
            df_drrs = df_drrs.append(data, ignore_index=True)
        elif f.find('Daily SpO2') != -1:
            data = pd.read_csv(full_path)
            df_daily_spo2 = df_daily_spo2.append(data, ignore_index=True)
        elif f.find('Device Temperature') != -1:
            data = pd.read_csv(full_path)
            df_device_temp = df_device_temp.append(data, ignore_index=True)
        elif f.find('Heart Rate Variability Details') != -1:
            data = pd.read_csv(full_path)
            df_hr_var = df_hr_var.append(data, ignore_index=True)
        elif f.find('Heart Rate Variability Histogram') != -1:
            data = pd.read_csv(full_path)
            df_hr_hist = df_hr_hist.append(data, ignore_index=True)
        elif f.find('Minute SpO2') != -1:
            data = pd.read_csv(full_path)
            df_min_spo2 = df_min_spo2.append(data, ignore_index=True)
        elif f.find('Respiratory Rate Summary') != -1:
            data = pd.read_csv(full_path)
            df_rrs = df_rrs.append(data, ignore_index=True)
        elif f.find('sleep-') != -1:
            with open(full_path, 'r') as f:
                json_data = pd.read_json(f)
            data = pd.DataFrame(json_data)
            df_sleep = df_sleep.append(data, ignore_index=True)
        elif f.find('sleep_score') != -1:
            data = pd.read_csv(full_path)
            df_sleep_score = df_sleep_score.append(data, ignore_index=True)
    return df_temp, df_var_summary, df_drrs, df_daily_spo2, df_device_temp, df_hr_var, df_hr_hist, df_min_spo2, df_rrs, df_sleep, df_sleep_score


In [22]:
def create_stress_dataFrames(path):
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    df4 = pd.DataFrame()
    for f in os.listdir(path):
        full_path = os.path.join(path, f)        
        if f.find('Mindfulness Eda Data Sessions') != -1 or f.find('mindfulness_eda_data_sessions') != -1:
            data = pd.read_csv(full_path)
            df1= df1.append(data, ignore_index=True)
        elif f.find('Mindfulness Sessions') != -1 or f.find('mindfulness_sessions') != -1:
            data = pd.read_csv(full_path)
            df2 = df2.append(data, ignore_index=True)
        elif f.find('Stress Score') != -1:
            data = pd.read_csv(full_path)
            df3 = df3.append(data, ignore_index=True)
        elif f.find('mindfulness_goals') != -1:
            data = pd.read_csv(full_path)
            df4 = df4.append(data, ignore_index=True)
    return df1, df2, df3, df4

Funkcje do agregowania DataFramów dla wszystkich uczestników i zapisywania ich do nowych plików.

In [23]:
def aggregate_data(path, wanted_folder, data_part=None):
    if wanted_folder == "Heart":
        data = create_heart_dataFrames(path)
        return data
    elif wanted_folder == "Other":
        data = create_other_dataFrames(path)
        return data
    elif wanted_folder == "Physical Activity" and data_part == 1:
        data = create_physical_activity_dataFrames_part1(path)
        return data
    elif wanted_folder == "Physical Activity" and data_part == 2:
        data = create_physical_activity_dataFrames_part2(path)
        return data
    elif wanted_folder == "Physical Activity" and data_part == 3:
        data = create_physical_activity_dataFrames_part3(path)
        return data
    elif wanted_folder == "Physical Activity" and data_part == 4:
        data = create_physical_activity_dataFrames_part4(path)
        return data
    elif wanted_folder == "Sleep":
        data = create_sleep_dataFrames(path)
        return data
    elif wanted_folder == "Stress":
        data = create_stress_dataFrames(path)
        return data

In [36]:
def clean_folders_and_aggregate_files(main_path, wanted_folder=None, files_names=[], data_part=None, print_files_list=False, cleansing=False):
    if len(files_names) > 0:
        # tworzenie pustych dataFramów
        for i in range(len(files_names)):
            globals()['df_{num}'.format(num=i+1)] = pd.DataFrame()
    files_list = []
    # iterowanie po folderach każdego użytkownika
    for iter in os.listdir(main_path):
        iter_path = os.path.join(main_path, iter)
        for user_folder in tqdm(os.listdir(iter_path)):
            user_path = os.path.join(iter_path, user_folder)
            for user_name_folder in os.listdir(user_path):
                user_name_path = os.path.join(user_path, user_name_folder)
                if user_name_folder.find('pdf') != -1:
                    # usuwanie pliku pdf
                    os.remove(user_name_path)
                else:
                    # wszystkie foldery dla użytkownika
                    for folder in os.listdir(user_name_path):
                        # iterowanie po wszystkich folderach użytkownika
                        folder_path = os.path.join(user_name_path, folder)
                        if cleansing:
                            remove_readme_files(folder_path)
                            remove_empty_files(folder_path)
                            remove_empty_folders(folder_path)
                        if folder == wanted_folder:
                            # obsługa plików
                            if len(files_names) > 0:
                                df_out = aggregate_data(folder_path, wanted_folder, data_part=data_part)
                                for i in range(len(files_names)):
                                    df = df_out[i]
                                    df["iteration"] = str(iter)
                                    df["userID"] = str(user_folder)
                                    globals()['df_{num}'.format(num=i+1)] = globals()['df_{num}'.format(num=i+1)].append(df, ignore_index=True)
                            files = return_files_list(folder_path)
                            files_list += files
    # zapisywanie danych do plików
    if files_names != []:
        # tworzenie nowego folderu fitbit_out
        path = main_path.split('fitbit')[0]+'fitbit_out/'
        if not os.path.exists(path):
            os.makedirs(path)
        
        for i in range(len(files_names)):
            path = main_path.split('fitbit')[0]+'fitbit_out/'+wanted_folder
            if not os.path.exists(path):
                os.makedirs(path)
            globals()['df_{num}'.format(num=i+1)].to_csv('{main_path}/fitbit_out/{folder_name}/{file_name}'.format(main_path=main_path.split("/fitbit_in")[0], folder_name=wanted_folder, file_name=files_names[i]), index=False)
    if print_files_list:
        return set(files_list)


In [ ]:
########## ROZPAKOWANIE FOLDERÓW ZIP ##########
unpack_zip_folder(main_path)

In [ ]:
########## CZYSZCZENIE WSZYSTKICH FOLDERÓW ##########
clean_folders_and_aggregate_files(main_path, cleansing=True)

# Agregacja wszystkich plików
Foldery wyjściowe:
- HEART
- OTHER
- PHYSICAL ACTIVITY
- SLEEP
- STRESS

## Folder HEART

In [ ]:
# wypisanie listy plików w folderze
clean_folders_and_aggregate_files(main_path, wanted_folder='Heart', print_files_list=True)

In [ ]:
clean_folders_and_aggregate_files(main_path, wanted_folder='Heart', files_names=['afib_ecg_reading.csv', 'hr_notifications_alerts.csv', 'hr_notifications_profile.csv'])

## Folder OTHER

In [ ]:
# wypisanie listy plików w folderze
clean_folders_and_aggregate_files(main_path, wanted_folder='Other', print_files_list=True)

In [ ]:
clean_folders_and_aggregate_files(main_path, wanted_folder='Other', files_names=['estimated_oxygen_variation.csv'])

## Folder PHYSICAL ACTIVITY

In [ ]:
# wypisanie listy plików w folderze
clean_folders_and_aggregate_files(main_path, wanted_folder='Physical Activity', print_files_list=True)

In [ ]:
clean_folders_and_aggregate_files(main_path, wanted_folder='Physical Activity', files_names=['active_zone_minutes.csv', 'activity_goals.csv', 'calories.csv', 'demographic_vo2_max.csv', 'distance.csv', 'exercise.csv'], data_part=1)

In [ ]:
clean_folders_and_aggregate_files(main_path, wanted_folder='Physical Activity', files_names=['heart_rate.csv', 'lightly_active_minutes.csv'], data_part=2)

In [ ]:
clean_folders_and_aggregate_files(main_path, wanted_folder='Physical Activity', files_names=['moderately_active_minutes.csv', 'resting_heart_rate.csv', 'sedentary_minutes.csv', 'steps.csv', 'swim_lengths.csv'], data_part=3)

In [ ]:
clean_folders_and_aggregate_files(main_path, wanted_folder='Physical Activity', files_names=['time_in_heart_rate_zones.csv', 'very_active_minutes.csv'], data_part=4)

## Folder SLEEP

In [ ]:
# wypisanie listy plików w folderze
clean_folders_and_aggregate_files(main_path, wanted_folder='Sleep', print_files_list=True)

In [ ]:
clean_folders_and_aggregate_files(main_path, wanted_folder='Sleep', files_names= ['computed_temperature.csv', 'daily_hr_variability_summary.csv', 'daily_respiratory_var_summary.csv',
                                                                                'daily_SpO2.csv', 'device_temperature.csv', 'heart_rate_variability_details.csv',
                                                                                'heart_rate_variability_hist.csv', 'minute_SpO2.csv', 'respiratory_rate_summary.csv',
                                                                                'sleep.csv', 'sleep_score.csv'])

## Folder STRESS

In [ ]:
# wypisanie listy plików w folderze
clean_folders_and_aggregate_files(main_path, wanted_folder='Stress', print_files_list=True)

In [ ]:
clean_folders_and_aggregate_files(main_path, wanted_folder='Stress', files_names=['mindfulness_eda_data_sessions.csv', 'mindfulness_sessions.csv', 'stress_score.csv', 'minfulness_goals.csv'])